# <center>Deepseek企业级Agent项目开发实战</center>

## <center>Part 7. 基于API驱动的Microsoft GraphRAG工程化架构实现</center>

&emsp;&emsp;通过前面六节课的学习，我们已经详细讲解了`Microsoft GraphRAG` 在构建索引和检索两个阶段时其底层运行的原理和各个`Workflow`的实现细节。在这个基础上，我们又进一步的对`Microsoft GraphRAG` 做了源码级的二次开发，主要实现如下优化策略：

1. 对`.csv`文件的索引流程做了优化， 不再使用简单的按照`token`数量切分，而是结合`csv` 文件的预处理流程，构建<font color=red>分层分级动态切分</font>，从而保证每个`row` 可以保留完整的语义；
2. <font color=red>自定义集成了支持并行解析的`MinerU`服务</font>，支持对`PDF`格式、图像（`.jpg`及`.png`）、`Word（.doc及.docx）`、以及`PowerPoint（.ppt及.pptx）`在内的多种文档格式的文档解析；
3. 支持对图像、表格类型数据的分析和处理，<font color=red>接入多模态（视觉）模型对图片和表格进行分析</font>，并生成对应的文本描述，从而保证`GraphRAG` 可以处理更多的数据类型；
4. 新增 `PDF` 等格式文件的文档切分器，通过<font color=red>结构感知动态分块策略</font>，有效保留文本和表格/图谱多模式语义组合长文本及图像、表格等单独语义的隔离；


&emsp;&emsp;上述各阶段的优化策略，其一是扩展了`Microsoft GraphRAG` 对数据类型的支持，针对实际业务中常见的数据类型，如`csv`、`PDF`、`图片` 等，都可以通过我们二次开发后的`Microsoft GraphRAG` 索引流程进行处理，并生成对应的图谱；其二是通过优化索引流程，在保证索引质量的前提下，显著提升了索引的效率，从而可以支持更大规模的数据处理。（`MinerU`的并行解析能力，并实现了服务分离）。而对检索来说，检索效果的好坏根本原因在于索引阶段构建的图谱质量，如果索引阶段构建的图谱质量不高，那么检索阶段无论使用什么样的检索策略，效果都不会太好。这一点是需要大家明确理解的。

&emsp;&emsp;因此，在大家实际基于`Microsoft GraphRAG` 作为核心架构构建本地知识库业务时，重心应该放在索引阶段，通过优化索引流程，构建高质量的图谱，从而保证检索阶段的效果。

&emsp;&emsp;而对于启动`Microsoft GraphRAG` 的索引和检索流程，在之前的实践中我们都是通过`Microsoft GraphRAG` 的`CLI` 命令行启动或者通过源码的`poetry` 命令启动。这两种不同的使用方法本质上的差异如下表所示：

<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>CLI 和 Poe 启动方式对比</font></p>
<div class="center">

| 特性         | CLI 启动方式                                   | Poe 启动方式                                   |
|--------------|-----------------------------------------------|------------------------------------------------|
| 安装方式     | pip 安装                                      | 源码安装                                       |
| 启动方式     | 在控制台输入命令启动                          | 在控制台输入命令启动                          |
| 可用参数     | 根据 `--help` 提供的参数指定构建索引/检索流程 | 可以任意添加参数，满足定制化的需求            |
| 灵活度       | 灵活度低，无法满足一些特殊的需求             | 灵活度高，可以修改源码，任意添加参数          |

</div>


&emsp;&emsp;这里需要注意的是：<**两种启动方式的共同点是都需要在控制台输入命令启动**。而通过控制台启动的索引构建、查询的流程如果对于离线构建索引和检索来说，是没有任何问题的。因为离线构建索引和检索通常都是一次性操作，不需要频繁调用。但是，对于需要频繁调用索引和检索的场景来说，这种方式就显得有些笨重了，而且是没有办法和业务系统进行融合的。比如用户在前端页面中点击某个按钮，如果每次都需要在控制台输入命令，那么这种应用方式显然是不合适的。


&emsp;&emsp;因此，在本节课中，我们将介绍如何通过`Python` 代码程序启动`Microsoft GraphRAG` 的索引和检索流程，并实际开发实现一个`Web`服务来展示`Microsoft GraphRAG` 的检索效果。同时，针对业务场景中的一些常见需求：如何处理多类型文本、如何做知识图谱更新等实现思路和优化方法，也将一一进行介绍。

&emsp;&emsp;我们首先来看如何通过`Python` 代码程序启动`Microsoft GraphRAG` 的索引构建流程。

&emsp;&emsp;在`Microsoft GraphRAG` 项目源码中，官方封装并提供了`index`、`query` 和 `prompt_tune` 三个流程的`API` 接口，我们可以直接通过`Python` 代码调用这些`API` 接口，从而实现索引构建、检索和提示词调优等流程。其源码存储位置为：`graphrag\graphrag\api` 目录下。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021343228.png" width=80%></div>

&emsp;&emsp; 其中`index.py` 中定义了`Microsoft GraphRAG` 的索引构建的`Pipeline` 接口，`prompt_tune.py` 中定义了提示词调优的`Pipeline` 接口，而`query.py` 中定义了检索的`Pipeline` 接口。根据我们之前的课程讲解，大家应该也能深刻理解到对于使用`Microsoft GraphRAG` 构建本地知识库来说，正确的流程就是先通过`prompt_tune` 根据实时实际的数据进行提示词自适应调优，然后再构建索引，最后才是通过`query` 检索。

&emsp;&emsp;因此，我们也按照这个顺序来给大家讲解如何自定义接入这三个`API`接口，从而完成代码层面的应用。首先来看`prompt_tune` 提示词调优的`API` 接口。

# 1. Prompt动态编排服务接口设计

&emsp;&emsp;在《Part 5: Microsoft GraphRAG 多源数据索引构建方案》 的 `2.4 实体关系提取优化策略` 小节中，我们给大家详细讲解了`Microsoft GraphRAG` 的提示词调优的实现原理和实现细节。在`ClI` 或者`Poetry` 命令行中可接入的命令参数在`API`的调用过程中同样适用。主要如下所示：


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>Auto Prompt Tuning 优化参数</font></p>
<div class="center">

| 参数名                     | 描述                                                                                     | 状态   | 默认值       |
|--------------------------|----------------------------------------------------------------------------------------|------|------------|
| --config                 | 配置文件的路径。这是加载数据和模型设置所必需的。                                            | 必需   | 无         |
| --root                   | 数据项目根目录，包括配置文件（YML、JSON 或 .env）。                                          | 可选   | 当前目录     |
| --domain                 | 与输入数据相关的域，例如“空间科学”、“微生物学”或“环境新闻”。如果留空，则将从输入数据中推断出域。 | 可选   | 无         |
| --selection-method       | 选择文档的方法。选项包括 all、random、auto 或 top。                                      | 可选   | random     |
| --limit                  | 使用随机或顶部选择时要加载的文本单元的限制。                                             | 可选   | 15         |
| --language               | 用于输入处理的语言。如果它与输入的语言不同，则将进行翻译。默认值为“”，表示将自动从输入中检测。 | 可选   | 自动检测 |
| --max-tokens             | 提示生成的最大令牌数。                                                                    | 可选   | 2000       |
| --chunk-size             | 用于从输入文档生成文本单元的标记大小。                                                  | 可选   | 200        |
| --n-subset-max           | 使用自动选择方法时嵌入的文本块数量。                                                    | 可选   | 300        |
| --k                      | 使用自动选择方法时要选择的文档数量。                                                    | 可选   | 15         |
| --min-examples-required   | 实体提取提示所需的最小示例数。                                                            | 可选   | 2          |
| --discover-entity-types   | 允许自动发现和提取实体。当数据涵盖大量主题或高度随机化时，建议使用此选项。           | 可选   | 无         |
| --output_path                 | 保存生成的提示的文件夹。                                                                  | 可选   | prompts |

</div>

&emsp;&emsp;当然，其对应的`API`接口无非就是将这些命令参数转换为`Python` 代码，并通过直接调用工具函数的形式构建出一个可以直接运行的`Pipeline`。其源码存储位置为：`graphrag\graphrag\api\prompt_tune.py`


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021524392.png" width=80%></div>

&emsp;&emsp; 其核心流程依然包含以下 11 个步骤：

1. 解析`settings.yaml` 中配置的参数， 通过 `--config` 参数；
2. 找到在 `input` 文件夹中存放的文档， 通过 `--root` 参数指定根目录；
3. 将读取到的文档切分成`text_unit` （块），通过 `--chunk_size` 参数指定切分策略中每个块的大小；
4. 限制用于后续生成提示的原始文档的数量，通过 `--limit` 参数指定；
5. 根据 `--k` 参数，进一步选择具体的`text_unit` 块，包含三个策略：
    - 如果 `--selection-method` 参数为 `top`，则选择前`k`个文本单元；
    - 如果 `--selection-method` 参数为 `random`，则随机选择`k`个文本单元；
    - 如果 `--selection-method` 参数为 `auto`，则使用`k`和`n_subset_max`选择文本单元，具体的实现逻辑是：
        - `--n-subset-max` 用来限制随机采样的取值，表示从`text_unit`个文本单元中随机采样`n_subset_max`个文本单元，转化成 `Embedding` 向量；
        - 计算采样文本单元的 `Embedding` 向量平均值，即中心点；
        - 计算每个采样文本块到中心点的距离，选择出`k`个距离中心点最近的文本单元；
        - 最终选择出来的文本最能代表整个文档集合的一般特征，同时可以过滤掉一些极端的文本块；
6. 设置输入数据所属的场景领域，比如`电商客服`、`算法分析`等，用于填充`Prompt` 模板中的`domain` 参数，如果未指定，则通过内置提示词 + 语言模型自动生成，最后填充 `Prompt` 模板中的`domain` 参数；
7. 设置最终输出的提示词模版的语言，通过`--language` 参数指定，如果未指定，则通过内置提示词 + 语言模型自动生成，最后填充 `Prompt` 模板中的`language` 参数；
8. 根据设置的领域`domain`，生成模型的系统身份背景信息；
9. 如果设置 `--discover_entity-types`， 则会通过提示词 + 语言模型自动发现符合当前输入数据的实体类型；
10. 生成用于生成实体配置的实体/关系示例列表，用来填充`Prompt` 模板中的`examples` 参数，即`Few-shot` 示例;
11. 最后，依次生成用于提取实体、关系、社区报告、摘要的提示词，可以通过 `--min_examples-required` 参数指定每个提示词需要的最小示例数量，以及通过`--output_path` 参数指定输出文件夹。

&emsp;&emsp;这里不再重复讲解，如不清楚实现细节，请观看《Part 5: Microsoft GraphRAG 多源数据索引构建方案》 的 `2.4 实体关系提取优化策略` 小节的视频讲解。我们直接来看如何通过`Python` 代码调用`prompt_tune` 的`API` 接口。根据该接口形式，在源码中新建了一个`dev` 文件夹，并在其中创建了`graphrag_prompt_tune.py` 文件，其源码存放位置为：`graphrag\dev\graphrag_prompt_tune.py`， 如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021533365.png" width=80%></div>

&emsp;&emsp;`run_prompt_tune` 函数是`graphrag_prompt_tune.py` 文件中的核心函数，其核心思路就是自定义加载一系列适用于`prompt_tune` 参数配置，传入到`prompt_tune` 的`API` 接口函数中的`generate_indexing_prompts`函数，从而实现`Microsoft GraphRAG` 中代码环境下运行提示词调优的需求。支持的配置文件如下所示：
```json
    # ========================================
    # 用户配置参数 - 直接修改这里的变量
    # ========================================

    # GraphRAG项目根目录路径
    PROJECT_DIR = "E:\\my_graphrag\\graphrag_2.1.0\\graphrag"

    # 数据目录名称（相对于项目根目录）
    DATA_DIR_NAME = "data"

    # 文档块大小
    CHUNK_SIZE = 500

    # 文档块重叠大小
    OVERLAP = 100

    # 要加载的文档块数量限制
    # 注意：如果文档较少，请设置较小的值
    LIMIT = 5

    # 文档选择方法: "random", "auto", "all", "top"
    # 参考DocSelectionType枚举的有效值
    SELECTION_METHOD = "random"

    # 领域（为None则自动检测）
    DOMAIN = None

    # 语言（为None则自动检测）
    LANGUAGE = None

    # 提示词最大令牌数
    MAX_TOKENS = 4000

    # 是否自动发现实体类型
    DISCOVER_ENTITY_TYPES = True

    # 最小所需的示例数
    MIN_EXAMPLES_REQUIRED = 2

    # 嵌入子集最大数量（对auto选择方法有效）
    N_SUBSET_MAX = 300

    # 文档选择数量（对auto选择方法有效）
    K = 15

    # 输出目录
    OUTPUT_DIR = "E:\\my_graphrag\\graphrag_2.1.0\\graphrag\\data\\prompt_turn_output"
```


&emsp;&emsp; 其中需要重点说明的就是：这里`PROJECT_DIR` 是`Microsoft GraphRAG` 项目的根目录，要填写服务器上实际存储的`Microsoft GraphRAG` 路径。而 `DATA_DIR_NAME` 是数据目录名称，也就是大家通过`poetry run poe init --root xxx` 命令初始化时指定的目录。而输出目录`OUTPUT_DIR`，则是配置用于存放微调后新提示词的文件夹。

&emsp;&emsp;同时需要注意的是：按照上述的参数规范自定义修改`prompt_tune` 的参数配置后，还需要理解的是：`prompt_tune` 无论是命令行调用，还是通过`Python` 代码调用，本质上加载的配置文件都是`settings.yaml` 文件，因此，大家也需要保证`settings.yaml` 文件中正确配置好了参数，其中比较关键的是：大模型连接信息、Embedding 模型的连接信息，以及要用于提示词调优的原始文档，即通过`input`下的参数进行过滤和选择。


&emsp;&emsp;大家可以参考如下的配置进行自定义修改。首先是`.env` 文件。在这个文件中，我们可以添加更多的环境变量以优化`settings.yaml` 文件中的参数配置。如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021557556.png" width=80%></div>

&emsp;&emsp; 然后是`settings.yaml` 文件。在这个文件中，我们可以添加更多的参数配置以优化`prompt_tune` 的参数配置。主要：这里通过 `${}` 的方式，将`.env` 文件中的环境变量引入到`settings.yaml` 文件中，因此只要修改了`.env` 文件中的环境变量，`settings.yaml` 文件中也会自动更新。 核心关注的是：`input` 下面要选择哪种格式的文档作为提示词调优的原始文档。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021603973.png" width=80%></div>

&emsp;&emsp; 如上所示，我们这里选择是`txt`的文件进行提示词调优的原始文档。同样，因仅用于功能测试，我们这里还是选择`technology_companies.txt`这篇短文档进行快速验证。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021604929.png" width=80%></div>

&emsp;&emsp;当配置好了`settings.yaml` 文件，同时也将`graphrag_prompt_tune.py` 文件中用于控制`prompt_tune` 的参数配置修改好后，则可以直接运行该`Python`脚本，执行代码层面的提示词调优流程。执行如下命令：

```bash
    cd dev
    python graphrag_prompt_tune.py
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021611307.png" width=80%></div>

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021615607.png" width=80%></div>

&emsp;&emsp;等待执行完成后，会将基于本地文档生成的提示词输出到`E:\my_graphrag\graphrag_2.1.0\graphrag\data\prompt_turn_output` 目录下，即在`graphrag_prompt_tune.py` 文件中通过`OUTPUT_DIR` 参数指定的输出目录。如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021616629.png" width=80%></div>

&emsp;&emsp;至此，我们就已经可以通过`Python` 代码执行`Microsoft GraphRAG` 的提示词调优流程。而准备好了适配私有数据文件的提示词后，在进行索引构建之前，记得将新生成的提示词设置为`settings.yaml` 文件中的默认提示词。如下所示：（如果使用`DeepSeek` 模型，建议大家先手动将提示词转化成中文后再进行设置，效果更佳。）


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021631546.png" width=80%></div>

&emsp;&emsp; 全部调整并配置好以后，接下来我们来看如何通过`Python` 代码执行`Microsoft GraphRAG` 的索引构建流程。


# 2. 索引构建的工程化接口封装

&emsp;&emsp;`Indexing` 的 `API` 接口源码文件存放在`graphrag\graphrag\api` 目录下的`index.py` 文件中，与`prompt_tune` 的`API` 接口类似，借助`index.py` 文件中定义`build_index`函数便可以实现通过`Python` 代码执行`Microsoft GraphRAG` 的索引构建流程。如下所示：

c<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504011528147.png" width=80%></div>

&emsp;&emsp;`index.py`中执行构建索引的`API` 接口核心就是在调用`index`模块中`run_pipeline`函数，本质上就是在启动`Microsoft GraphRAG` 的索引构建`Workflow`。调用关系如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021351048.png" width=80%></div>

&emsp;&emsp;关于`Microsoft GraphRAG`构建索引的完整`WorkFlow`流程，在《Part 2. Microsoft GraphRAG 索引构建细节源码详解》 中有非常详细的讲解，这里不在重复说明。我们需要做的是：基于这个`API` 接口去构建`Python`的接口调用程序。因此我们在源码文件中创建了一个新的文件夹`dev`，并且创建了`graphrag_indexing.py`文件，用于实现`Microsoft GraphRAG` 的索引构建流程，其源码存放位置为：`graphrag\dev\graphrag_indexing.py`


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021436473.png" width=80%></div>

&emsp;&emsp; 在`graphrag_indexing.py`文件会加载一系列变量以控制索引构建过程的灵活性，其中核心配置项如下：

```json
    # ========================================
    # 用户配置参数 - 直接修改这里的变量
    # ========================================

    # GraphRAG项目根目录路径
    PROJECT_DIR = "E:\\my_graphrag\\graphrag_2.1.0\\graphrag"

    # 数据目录名称（相对于项目根目录）
    DATA_DIR_NAME = "data"

    # 索引方法: "Standard" 或 "Fast"
    INDEX_METHOD = "Standard"

    # 是否进行增量更新
    IS_UPDATE = False

    # 是否进行内存分析
    MEMORY_PROFILE = False

    # 配置文件路径（为None则使用默认配置, 如果是增量更新，则需要指定配置文件，比如："E:\\my_graphrag\\graphrag_2.1.0\\graphrag\\data\\settings_pdf.yaml"）
    CONFIG_FILE = None

    # 输出目录（为None则使用配置中的默认输出目录，）
    OUTPUT_DIR = None
```


&emsp;&emsp;其中需要重点说明的就是：这里`PROJECT_DIR` 是`Microsoft GraphRAG` 项目的根目录，要填写服务器上实际存储的`Microsoft GraphRAG` 路径。而 `DATA_DIR_NAME` 是数据目录名称，也就是你通过`poetry run poe init --root xxx` 命令初始化时指定的目录。`IS_UPDATE` 参数为启动增量更新的配置项，大家可以暂时忽略，我们下一小节重点给大家进行介绍。这里先设置为`False`表示不进行增量更新。

&emsp;&emsp;核心代码逻辑请观看视频讲解。大家按照上述的参数规范自定义修改后，则可以直接运行该`Python`脚本，同时需要说明的是：这个执行过程加载的配置文件依然是`settings.yaml` 文件。执行命令如下：

```bash
    python graphrag_indexing.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021646214.png" width=80%></div>

&emsp;&emsp;等待执行完成后，会与 `CLI` 或者 `Poetry` 命令行执行的流程一样，在输出路径下会依次生成`.cache`、`logs`和`output` 三个文件夹，分别用于存储索引构建的缓存数据、日志信息和索引构建的输出结果。如下图所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021717428.png" width=80%></div>

&emsp;&emsp; 其中，`logs` 文件夹中会生成`dev_graphrag_indexing.log` 文件，用来记录通过`Python` 代码执行索引构建的日志信息。至此，我们就可以通过`Python` 代码执行`Microsoft GraphRAG` 的索引构建了。正常来说，执行完这一步便可以进行`query` 检索了。但在此之前，我们需要进一步给大家介绍下关于实际构建索引过程中遇到的常见需求及对应的解决方案。


&emsp;&emsp; <font color=red>**企业常见需求一：如何构建多个索引？**</font>

&emsp;&emsp; 多索引构建对应的开发场景大多存在于企业中存在多个数据源支撑多个业务线，彼此之间相对独立，且每个业务线之间存在一定的差异性，比如：数据格式、数据内容、数据规模等，冗余在一起会加重检索的效率和维护成本。（因为知识图谱越大，检索时需要检验的信息就越多，检索的效率就越低），亦或对不同的部门、人员之前的知识库进行隔离，避免知识库之间的相互污染等常见问题。





&emsp;&emsp; 这类问题对应的解决方案在当前的流程中非常容易实现：即只需要在构建的时候，通过`init`的`--root`参数指定不同的数据源根目录，制定索引文件与数据源的对应关系（如通过字典构建映射关系），形式如下：

```python
    # 索引文件与数据源的对应关系
    INDEX_FILE_MAP = {
        "index_1": "data_source_1",
        "index_2": "data_source_2",
    }
```

&emsp;&emsp;各个索引知识库独立构建，在进行问答检索时先通过`INDEX_FILE_MAP`字典找到对应的索引文件根路径，进行加载即可实现多知识库的隔离检索。

&emsp;&emsp;<font color=red>**企业常见需求二：多个不同类型的数据源，如何共同创建一个索引？**</font> 

&emsp;&emsp;多数据源共建一个知识库的场景，在企业中也比较常见。比如关系型数据库、`PDF`、`Word`、`Excel`、`CSV` 等不同类型的数据源，希望各个数据源中的有效信息可以实现整合，从而构建一个统一的知识库提供用户、或者下游的智能体使用。

&emsp;&emsp;回顾`Microsoft GraphRAG` 的索引构建流程，筛选不同文件进入构建索引的`Workflow` 流程，核心是在`settings.yaml` 文件中`input` 参数下借助`file_type` 参数指定文件类型，以及`file_parttern` 通过正则表达式的形式过滤符合标准的文件，比如通过(`.*\.txt$`) 过滤所有`txt` 文件。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021738213.png" width=80%></div>

&emsp;&emsp;那么对于这种过滤文件的方式，大家可能想到的是：既然借助`file_type` 和`file_parttern` 参数可以过滤文件，那么是否可以借助`file_type` 和`file_parttern` 参数，将不同类型的数据源进行整合，从而构建一个统一的知识库？ 实际上确实可以的，比如指定`file_pattern: ".*\\.(txt|csv)$$"` ,即可同时读取`input` 目录下所有`txt` 和`csv` 文件。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021744525.png" width=80%></div>

&emsp;&emsp;从执行流程上看是可以，但是存在一个非常严重的问题是：无法指定不同文件类型加载对应的优化切分策略。也就是说，`input` 可以通过 `file_type` 和 `file_pattern` 参数指定文件类型和文件过滤规则，但是`chunk`下的切分策略，将会用于所有文件的切分，也就是说无法实现不同文件类型加载对应的优化切分策略。 比如我们针对不同的文件格式，定义了优化后的切分策略，配置如下所示：

```json
    input:
    type: file # or blob
    file_type: csv
    base_dir: "input"
    file_encoding: utf-8
    file_pattern: ".*\\.csv$$"
    text_column: "text"  # 如果原始数据中没有text字段，则需要指定一个包含文本内容的字段

    chunks:
    strategy: csv  # 自定义参数
    size: 500
    overlap: 100
    group_by_columns: [id]


    input:
    type: file # or blob
    file_type: pdf # or csv
    base_dir: "input"
    file_encoding: utf-8
    file_pattern: ".*\\.pdf$$"
    local_output_dir: "./data/pdf_outputs"  # 自定义参数：添加PDF输出目录配置
    mineru_api_url: "http://192.168.110.131:8000/"
    mineru_output_dir: "/home/07_minerU/tmp/"
    table_description_api_key: "sk-10d0917bf23e74592d54cac"
    table_description_model: "deepseek-chat"
    base_url: "https://api.deepseek.com"

    image_description_api_key: "sk-proj-CUT84SGBkYSEyqELt6itYDlmndxz2LGCrFFX9Oe0o170A"
    image_description_model: "gpt-4o"
    image_description_base_url: "https://ai.devtool.tech/proxy/v1"


    chunks:
    strategy: markdown  # 自定义参数
    size: 500
    overlap: 100
    group_by_columns: [id]
```


&emsp;&emsp;在现有流程上，是不能实现对文件格式和对应的切分策略进行灵活匹配的。因此，这里能得出的结论是：<font color=red>如果大家对所有格式的文件能够容忍使用同一种最简单的切分策略，那么可以直接用`file_pattern`进行文件过滤。而如果希望针对不同文件类型，加载对应的优化切分策略，则需要引入增量更新的解决方案。</font>


&emsp;&emsp;<font color=red>增量更新（Incremental Update）指的是在已有知识图谱的基础上，快速更新新加入的知识，避免全量重建。</font> 一些核心的增量更新的实现方法是：

1. **文档级增量更新**：这类方法会对文本内容生成MD5哈希值，在实现增量插入功能时，先检查是否存在相同内容，如果判定为新内容，则重新计算社区结构并生成新的社区报告。代表项目是`nano-GraphRAG`。
2. **知识库级增量更新**：这类方法会使用图结构管理实体和关系，每次有新文档加入时，会对其执行相同的图索引步骤，包括实体识别和关系提取。通过将新增的节点和边集与原有图的节点和边进行合并（集合并操作），实现新数据的无缝集成，代表项目是`LightRAG`。


&emsp;&emsp; `Microsoft GraphRAG` 在 v0.4.0 版本中引入了增量索引支持，其实现的方法属于<font color=red>**知识库级增量更新**</font>。在进行原理讲解之前，我们可以通过`Poetry` 命令直接看一下`Microsoft GraphRAG` 的增量索引的执行方法，如下所示，对增量更新，需要执行的命令为`poetry run poe update`， 可以通过`--help` 参数查看具体的执行方法，如下所示：


<style>
.center 
{
  width: auto;
  display: table;
  margin-left: auto;
  margin-right: auto;
}
</style>

<p align="center"><font face="黑体" size=4>增量更新参数说明</font></p>
<div class="center">

| 参数名                | 说明                                                                                     | 默认值         |
|---------------------|----------------------------------------------------------------------------------------|--------------|
| `--config`          | 要使用的配置文件路径。                                                                  | `None`       |
| `--root`            | 项目的根目录。                                                                         | `.`          |
| `--method`          | 要使用的索引方法。可选值为 `standard` 或 `fast`。                                       | `standard`   |
| `--verbose`         | 以详细日志运行索引管道。                                                               | `no-verbose` |
| `--memprofile`      | 运行索引管道时进行内存分析。                                                            | `no-memprofile` |
| `--logger`          | 要使用的进度日志记录器。可选值为 `rich`、`print` 或 `none`。                          | `rich`       |
| `--cache`           | 使用LLM缓存。                                                                          | `cache`      |
| `--skip-validation` | 跳过任何预检验证。适用于不运行LLM步骤时。                                              | `no-skip-validation` |
| `--output`          | 索引管道输出目录。覆盖配置文件中的 `output.base_dir`。                                 | `None`       |
| `--help`            | 显示此帮助信息并退出。                                                                  |              |
</div>


&emsp;&emsp;既然是增量更新，那么就意味着需要将新文件生成的索引文件，与已经存在的某一个图谱进行合并，因此，在执行`poetry run poe update` 命令时的核心参数为：

1.  `--config`: 指定新文件的默认加载配置，加载这个 `settings.yaml` 做文件切分等策略；

2. `--output`: 指的是要和哪一个已经存在的图谱进行合并， 如果不指定，就是 `--root` 文件夹中对应的 `output` 下面的 `.parquet` 文件；

3. `--vector_store`: 这个参数并没有在命令行参数中体现，是实际运行时总结出的一个关键点，即： `db_uri` 要与 `--output` 指定的图谱路径一致，否则会加载不到新合并的实体`Embedding`，导致合并后检索异常；

&emsp;&emsp;因此，我们这里可以通过`poetry run poe update` 命令，实践一下增量更新的执行流程。接下来我们将`input` 目录下的`csv` 文件，与刚刚对`txt` 文件构建的索引进行合并。 首先，修改`settings_csv.yaml` 文件，让其加载`input` 目录下的`csv` 文件，并使用`csv` 的切分策略，与上一小节对`txt` 文件构建的索引进行合并。配置修改如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021901999.png" width=80%></div>

&emsp;&emsp; 然后执行增量更新流程，执行命令如下，耐心等待执行结束即可。

```bash
    poetry run poe update --root ./data
```


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021905760.png" width=80%></div>

&emsp;&emsp;当增量更新执行结束后，发生的改动主要体现在：

1. 旧的索引文件会进行更新，融入新文件产生的实体、关系、描述等信息，因此可以看到就得索引的日期会发生变化，标志着索引文件已经更新。
2. 当指定的根路径下（课件中使用的是`./data`），会生成一个`updata_output` 文件夹，同时以时间戳命名的文件夹，该文件夹中里面会包含`delta` 和`previous` 两个子文件夹，分别用于存储新文件产生的实体、关系、描述等信息，以及旧文件的备份。


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504021934976.png" width=80%></div>

&emsp;&emsp; 至此，整个增量更新就已经执行结束了。该过程已经成功的将`merged_review.csv` 文件中的内容，与第一次构建`technology_companies.txt` 文件的索引进行了合并，其存储的索引文件依然是`output` 文件夹下的所有`.parquet` 文件。整个过程在命令行的操作形式下非常简单，只需要修改`settings.yaml` 文件，指定新的数据源，然后执行`poetry run poe update` 命令即可。而对于如果我们想将其封装成`Python` 接口，则需要进一步了解其背后的实现原理。因为在`Microsoft GraphRAG` 的`api` 接口中，仅仅提供了`prompt_tune.py`、`query_api.py` 和`index.py` 三个文件，并没有提供增量更新的接口，同时对于增量更新后新生成的索引文件应该如何使用，只有通过进一步的源码分析才能够真正理解其应用的方法。

&emsp;&emsp; 因此接下来，我们就详细解读`Microsoft GraphRAG` 中实现增量更新的完整底层逻辑，以支撑接下来通过`Python` 接口的开发实现增量更新功能的实际需求。


&emsp;&emsp; 既然可以通过`Poetry run poe update` 命令行执行增量更新，那么就意味着在`Microsoft GraphRAG` 的`CLI` 中，一定存在对应的实现方法。因此接下来，我们需要从`Microsoft GraphRAG` 的`CLI` 入手，源码位置为：`graphrag\graphrag\cli\main.py`

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031011113.png" width=80%></div>

&emsp;&emsp; 在`_update_cli` 函数中，`update_cli` 函数是用于执行增量更新的入口函数，根据执行逻辑，其本质上执行的还是`index` 的流程，只不过在执行的过程中，会根据增量更新的需求，进行一些特殊处理。路由逻辑如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031017403.png" width=80%></div>

&emsp;&emsp;因此增量更新的逻辑是：通过`is_update_run` 参数，判断是否执行增量更新，如果是`True` 则执行增量更新，否则执行正常的索引构建流程。其中增量更新的具体流程为：

1. **加载用于增量更新的文件**：根据在`input`中存放的文件， 通过`create_input`函数按照`settings.yaml`中配置的文档类型进行加载；
2. **读取已存在的`documents.parquet` 文件**：获取之前所有已经执行过索引的文件信息， 返回一个`DataFrame`对象，包括 `id`、`title`、`text` 和 `creation_date` 等字段；
3. **对比增量更新文件与已存在的`documents.parquet` 文件，具体的检索方法为**：
   - 先根据 `title` 去重 原始文档，得到 一个 `Dataframe`;
   - 再根据 `title` 去重 新增文档，得到 一个 `Dataframe`;
   - 检查新增文档中都有哪些是在原始文档中的，  返回一个包含所有新增文档的 `DataFrame`;
   - 检查原始索引文件中的每个标题是否在新数据集中， 返回一个包含所有需要删除文档的 `DataFrame`;

&emsp;&emsp;这个预处理的流程本质上就是在已有的文档索引中添加新的文档，同时删除不再需要的文档。而根据源码的解析，能够判断出来的一个关键点是：<font color=red>增量更新对不同文件的识别是根据`title` 字段进行的，而`title`字段对应的是`input` 目录下文件的名称，所以如果你要增量更新的文档名称与源文档是一样的话，即使内容发生了改变，也不会触发增量更新，因此一定要确保要执行的增量更新文档名称要进行修改，最好的一种方式就是以时间戳命名，比如`merged_review_20250403.csv`。</font>

&emsp;&emsp;接下来，当识别到了新增的文档后继续执行的流程为：

4. 创建一个用于存储新增文档的存储对象;
5. 创建带时间戳的子存储，（即创建本地文件夹），包含两次操作：
   - 在时间戳目录下创建 `delta` 子目录，用于存储新增的文档数据
   - 在时间戳目录下创建 `previous` 子目录， 用于存储原有数据的备份，包含所有的`.parquet`文件

6. 将新文档的 `storage` 和 `input` ，进入`Microsoft GraphRAG`的`Indexing` 的 `Workflow`， 先对新文档正常执行索引构建流程， 得到新文档对应的索引文件；

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031035877.png" width=80%></div>

&emsp;&emsp;这一步所做的事情就和正常 `poetry run poe index` 构建文件的索引没有任何的区别。我们对`Index`在《Part 2. Microsoft GraphRAG 索引构建细节源码详解》中有详细的介绍，这里就不再赘述了。这里需要注意的是，对于增量更新来说，其本质是把新文件的索引文件增加到已有的索引文件中，所以在执行结束后，会进行索引合并，如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031039426.png" width=80%></div>

&emsp;&emsp;`update_dataframe_output`函数是增量更新的核心处理逻辑，主要包括了以下流程：

7. 首先加载 `delta` 和 `previous` 文件夹下的 `parquet` 文件；
8. 从旧索引文件中，根据 `human_readable_id` 生成新文件的初始`human_readable_id`索引，用于解决新旧数据合并时，`human_readable_id` 冲突的问题；
9. 使用`np.arange` 生成连续的 `ID` 序列， 进行合并，旧数据在前，新数据在后，形成一个完整的`DataFrame` 对象；
10. 执行实体合并操作，具体流程为：
   - 依次加载旧实体和新的实体（`entities.parquet`）
   - 根据新旧数据的`human_readable_id` 进行拼接，合并的同时解决冲突，具体策略如下：
      - 基于 `title` 字段合并新旧实体， 创建新旧`ID`的映射关系 {新`ID`: 旧`ID`}
      - 对于 `id`、 `type`、 `human_readable_id`、`degree`、`x`、`y` 采用"保留首个"策略
      - 对 `description`、 `text_unit_ids` 采取列表合并策略
      - 对 `frequency` 是基于合并后的文本单元`ID`数量重新计算频率

11. 执行关系合并操作，具体流程为：
   - 依次加载旧关系和新关系（`relationships.parquet`）
   - 根据新旧数据的`human_readable_id` 进行拼接，合并的同时解决冲突，具体策略如下：
     - 按源和目标分组：`groupby(["source", "target"])`；
     - 对于 `id`、 `human_readable_id` 采用 "保留首个"策略；
     - 对 `description`、 `text_unit_ids` 采取列表合并策略；
     - `weight` 取权重的平均值；
     - `combined_degree` 组合度数动态更新， 重新计算每个节点的连接度数，更新源节点和目标节点的度数，最总计算结果是 为计算组合度数（源度数 + 目标度数）；

12. 对合并后的实体和关系，重新生成描述，具体流程为：
   - 实体根据`title` 进行分组进行`description` 的合并；
   - 关系根据`source` 和`target` 进行分组进行`description` 的合并；
13. 根据 {新ID: 旧ID} 的映射，更新新文本单元中的实体引用，对每个文本单元的 `entity_ids` 列表进行处理，如果`ID`在映射中存在，则替换为映射后的`ID`，如果`ID`不在映射中，保持原值；
14. 合并新旧社区及父社区的引用；
15. 合并社区报告；
11. 合并文本嵌入;

&emsp;&emsp;综上一共16个步骤进行增量更新，而从其源码的实现来看，`Microsoft GraphRAG` 的增量更新是属于`Knowledge Graph` 的增量更新。


&emsp;&emsp;至此，我们已经了解了`Microsoft GraphRAG` 的增量更新流程，得到的一个重要结论就是：<font color=red>增量更新只是`index` 流程中通过`is_update_run` 参数进行控制的一个分支。</font>因此接下来，我们再来看增量更新在`Python`接口中的实现就能够非常容易理解。


&emsp;&emsp;现在回到我们自定义的`generate_index` 函数，在`index` 流程中，会根据`IS_UPDATE` 参数，判断是否执行增量更新，如果执行增量更新，则同时配置新索引对应的配置文件，即`settings_update.yaml` 文件，即可快速实现代码调用层面的增量更新流程，源码如下图所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031148356.png" width=80%></div>

&emsp;&emsp; 这里我们可以进行实际的调用测试。其中需要修改的配置参数为：

1. `generate_indexing.py` 文件中的`IS_UPDATE` 参数， 设置为`True`；
2. `generate_indexing.py` 文件中的`CONFIG_FILE` 参数， 设置为新增文件索引对应执行的`settings_update.yaml` 配置；（如下我们使用的是`csv` 文件，所以配置文件为`settings_csv.yaml`）

3. 正常配置`settings_pdf.yaml` 文件；

&emsp;&emsp; 同样，在进行增量更新时，仍然需要使用`Auto Prompt Tuning` 进行`prompt` 的自动优化，修改配置如下：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031501104.png" width=80%></div>

&emsp;&emsp; 同时需要注意：如果使用的是`MinerU` 服务解析`PDF` 文件，需要先进行`MinerU` 服务的启动，否则会报错。完成上述修改后，直接在命令行执行`python generate_prompt_turn.py` 命令，执行`Auto Prompt Tuning` 流程，得到优化后的`prompt` 文件。然后根据实际的情况进行`settings_csv.yaml` 文件中对应提示模版的修改，这里不再重复演示。

&emsp;&emsp; 最后，对`generate_indexing.py` 文件进行修改， 配置`IS_UPDATE` 参数， 设置为`True`， 指定`CONFIG_FILE` 参数， 设置为新增文件索引对应执行的`settings_update.yaml` 配置完成上述修改后，直接在命令行终端执行`python generate_indexing.py` 命令，即可实现增量更新。如下代码所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031459792.png" width=80%></div>

&emsp;&emsp;等待执行完成后，会将新文件（`.csv`）的索引文件，与`output` 文件夹下的所有`.parquet` 文件进行合并，形成新的索引文件。即完成不同文件类型的增量更新。在进行问答检索时，可以同时获取到`csv` 文件和`txt` 文件的索引信息，实现知识图谱的跨文件类型检索。

&emsp;&emsp; 以上就是两种在实际项目中对`Microsoft GraphRAG` 离线构建索引针对不同文件类型构建同一索引及构建多文件索引的具体解决思路。当然，如果想在构建多索引的同时也能实现跨文件存储，则把这两种方案结合应用即可。


&emsp;&emsp;而接下来，我们进入最后一个环节，`Query` 流程的`Python` 接口实现。

# 3. 检索服务的工程化接口封装

&emsp;&emsp;`Microsoft GraphRAG` 的`Query` 一共实现了四个检索方法，分别是`local_search`、`global_search`、`drift` 和`basic`。其中`local_search` 和`global_search` 检索的底层原理在《Part 4. Microsoft GraphRAG Query构建流程详解》中有详细的介绍，其中`Local Search` 是基于实体的检索，`Global Search` 是基于社区的检索。除此以外，我们再给大家介绍两个新的检索方法，分别是`drift` 和`basic`。

&emsp;&emsp;对于`Basic` 检索方法，其实现的检索方法非常简单，核心流程如下：

1. 获取到所有的`text_units`;
1. 提取对应描述的 `Embedidng` 向量；
2. 将用户的问题与每一个`text_units` 的描述进行相似度计算，得到相似度最高的`text_unit`；
3. 返回最终的前 `K` 个检索结果。

&emsp;&emsp;而 `Drift`, 是对本地检索的一种优化，核心是为基于实体检索的`Local Search` 赋予局部全局检索的能力。它的具体做法是：

1. 依次获取`communities`, `community_reports`, `text_units`, `relationships`, `entities` 数据信息；
2. 获取本地存储的对应`Embedding` 向量描述；
3. Primer： 提交查询时，`DRIFT` 会将其与语义最相关的前 `K` 个社区报告进行比较，生成一个初始答案以及几个后续问题，这些问题充当全局搜索的简化版本，具体逻辑如下：
   - 从前`K` 个社区报告中列表中随机选择一个社区报告;
   - 借助大模型根据模板扩展原始查询，生成更丰富的查询表示;
   - 通过嵌入模型将扩展后的查询转换为向量表示；
   - 将社区报告分割成多个部分，实现并行处理；
   - 根据社区报告中的全局知识，将查询分解成多个子查询；
   - 每个子查询关注不同的维度或方面，对每个报告分组异步执行查询分解；
   - 合并所有子查询的处理结果；

4. Follow-Up：有了`Primer` 过程生成子问题与对应的结果，后，使用本地搜索执行每个后续行动。产生额外的中间答案和后续问题，从而创建一个不断完善的循环，直到搜索引擎满足其终止标准的自动执行过程；
5. Output Hierarchy:：输出层次结构，最终输出是问题和答案的层次结构，按其与原始查询的相关性排序。

&emsp;&emsp;对这四种不同的检索方法，在`Microsoft GraphRAG` 提供的源码中的`api` 接口中，都提供了对应的实现方法，具体位置为：`graphrag\api\query_api.py` 文件中，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031421328.png" width=80%></div>


&emsp;&emsp; 因此按照相同的方式，我们就可以实现`Query` 流程的`Python` 接口实现。对应实现的源码文件为：`graphrag\dev\graphrag_query.py` 文件中，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031424298.png" width=80%></div>


&emsp;&emsp; 这里需要修改的参数还是一样，`PROJECT_DIR` 是`Microsoft GraphRAG` 项目的根目录，要填写服务器上实际存储的`Microsoft GraphRAG` 路径。而 `DATA_DIR_NAME` 是数据目录名称，然后根据问答情况灵活选择查询问题`QUERY`, 检索类型`QUERY_TYPE` 等灵活参数。执行的方法也非常简单，直接在命令行终端执行`python graphrag_query.py` 命令，即可实现`Query` 流程的`Python` 接口实现。如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031525251.png" width=80%></div>


&emsp;&emsp;至此，我们已经实现了`Microsoft GraphRAG` 的`Query` 流程的`Python` 接口实现，大家可以配置不同的问题类型进行实际的调用测试。


&emsp;&emsp; 本节课程内容的最后，我们在基于`Index`和 `Query` 流程的`Python` 接口实现的基础上，进一步将其封装成 `RESTful API` 接口，实现与外部系统的实时对接交互。


# 4. 基于FastAPI的RESTful端点设计

&emsp;&emsp; 


&emsp;&emsp;在实现了`Index` 和`Query` 流程的`Python` 接口后，其实就可以基于一些`Python` 框架，实现`RESTful API` 接口的封装，从而提供后端的服务与前端进行对接。这里我们选择`FastAPI` 框架，来介绍如何实现`RESTful API` 接口的封装。

&emsp;&emsp; 在`dev` 目录下已经实现了一个`graphrag_api.py` 文件，该文件中已经实现了`Query` 流程的`Python` 的外部接口封装，如下所示：

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031607435.png" width=80%></div>


&emsp;&emsp; 这个代码中只有一个地方需要大家根据自己的实际情况进行修改，即`start()` 函数中的 `host` 和 `port` 参数，需要根据实际的网络环境进行修改。

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031609337.png" width=80%></div>


&emsp;&emsp; 修改完成后，首先在命令行终端安装必要的依赖包，再执行`python graphrag_api.py` 命令，即可启动`RESTful API` 接口服务。如下所示：

```bash
    pip install fastapi uvicorn pydantic_settings
    python graphrag_api.py
```

<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031612860.png" width=80%></div>


&emsp;&emsp;启动服务后，则可以在`http://localhost:8000` 地址访问`GraphRAG`的前端页面，进行网页端的用户交互。如下所示：


<div align=center><img src="https://muyu20241105.oss-cn-beijing.aliyuncs.com/images/202504031622692.png" width=80%></div>
